numpy==1.24.3 
opencv==4.9.0.80 
onnxruntime==1.17.1 

In [1]:
import cv2
import numpy as np
import onnxruntime

In [2]:
path = "C:\\Users\\aimlc\\OneDrive\\Desktop\\Vidhul\\StonePaperScissor\\yolov5\\Rock-Paper-Scissors-SXSW-14\\test\\images\\IMG_7077_MOV-106_jpg.rf.70485649471c0d185b71d8061224b4b6.jpg"
im0 = cv2.imread(path)  # BGR
imcpy = cv2.copyMakeBorder(im0, 0,0,0,0, cv2.BORDER_CONSTANT, value=(114,114,11))
imcpy = imcpy.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
imcpy = np.ascontiguousarray(imcpy,dtype=np.float32)

In [3]:
im = imcpy
if len(im.shape)==3:
    im= im[None]

In [4]:
cuda = False
w=str('../runs/train/exp19/weights/best.onnx')
providers = ["CUDAExecutionProvider", "CPUExecutionProvider"] if cuda else ["CPUExecutionProvider"]
session = onnxruntime.InferenceSession(w, providers=providers)
output_names = [x.name for x in session.get_outputs()]
meta = session.get_modelmeta().custom_metadata_map  # metadata
if "stride" in meta:
    stride, names = int(meta["stride"]), eval(meta["names"])
y = session.run(output_names, {session.get_inputs()[0].name: im})[0]

In [5]:
outcpy=np.zeros((y.shape[1],6),dtype=y.dtype)
outcpy.shape

(25200, 6)

In [6]:
outcpy[:,0]=y[0,:,0]- y[0,:,2]/2
outcpy[:,1]=y[0,:,1]- y[0,:,3]/2
outcpy[:,2]=y[0,:,0]+ y[0,:,2]/2
outcpy[:,3]=y[0,:,1]+ y[0,:,3]/2
outcpy[:,5]=np.max(y[0,:,5:],axis=1)
outcpy[:,4]=y[0,:,4]*outcpy[:,5]
outcpy[:,5]=np.argmax(y[0,:,5:],axis=1)

In [7]:
out= outcpy[outcpy[:,4].argsort()[::-1]]
out.shape

(25200, 6)

In [8]:
def nms(boxes, overlap_threshold=0.45, min_mode=False):
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    scores = boxes[:, 4]
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    index_array = scores.argsort()[::-1]
    keep = []
    while index_array.size > 0:
        keep.append(index_array[0])
        x1_ = np.maximum(x1[index_array[0]], x1[index_array[1:]])
        y1_ = np.maximum(y1[index_array[0]], y1[index_array[1:]])
        x2_ = np.minimum(x2[index_array[0]], x2[index_array[1:]])
        y2_ = np.minimum(y2[index_array[0]], y2[index_array[1:]])

        w = np.maximum(0.0, x2_ - x1_ + 1)
        h = np.maximum(0.0, y2_ - y1_ + 1)
        inter = w * h

        if min_mode:
            overlap = inter / np.minimum(areas[index_array[0]], areas[index_array[1:]])
        else:
            overlap = inter / (areas[index_array[0]] + areas[index_array[1:]] - inter)

        inds = np.where(overlap <= overlap_threshold)[0]
        index_array = index_array[inds + 1]
    return keep

In [9]:
conf_thresh = 0.25
out = out[out[:,4]>conf_thresh]
out.shape

(299, 6)

In [10]:
keep=nms(out)
len(keep)

122

In [11]:
output=out[keep]
output.shape

(122, 6)